In [1]:
from fenics import *

In [ ]:
NX = 10
NY = 10

mesh = UnitSquareMesh(NX, NY)


# Problem setup
if SUPS is False:
    P1v = VectorElement("Lagrange", mesh.ufl_cell(), 2)
    parameters["form_compiler"]["quadrature_degree"] = 3
else:
    P1v = VectorElement("Lagrange", mesh.ufl_cell(), 1)
    parameters["form_compiler"]["quadrature_degree"] = 1
P1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
TH = MixedElement([P1v, P1, P1])
W = FunctionSpace(mesh, TH)
vp = TrialFunction(W)
(v, p, c) = split(vp)
(w, q, qc) = TestFunctions(W)
vp_ = Function(W)
v_function, p_function, c_function = split(vp_)
u_init = InitialConditionsNS()
vp_.interpolate(u_init)
c_function = vp_.split(deepcopy=True)[2]
c_function.interpolate(InitialConditionsCD())
out_c = File(DIR + "initialconcentration.pvd", "compressed")
out_c << (c_function, T)
vn = Function(W.sub(0).collapse())
cn = Function(W.sub(2).collapse())
cn = c_function
bcs = Lock_Exchange_boundary_condition(W, WIDTH, HEIGHT)
dtt = Constant(DT)
h = CellDiameter(mesh)
vnorm = sqrt(dot(v_function, v_function))